In [ ]:
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable
from torch import nn

%matplotlib inline

In [ ]:
class GenomesDataset(Dataset):
    def __init__(self, img_arr):
        '''img_arr is an array of images
            assuming they all have the same shape
            and structure channel-width-heigth'''
        self.img_arr = img_arr
        self.ch, self.w, self.h = img_arr[0].shape
    
    def __len__(self):
        return len(self.img_arr)
    
    def __getitem__(self, idx):
        return self.img_arr[idx]

In [ ]:
genome_dataset = GenomeDataset()

In [ ]:
genome_dataloader = DataLoader(genome_dataset, batch_size=32,
                                shuffle=True)

In [ ]:
class ProjectNetwork(nn.Module):
    def __init__(self, input_size, input_channels):
        super(ProjectNetwork, self).__init__()
        self.conv_layers = nn.Sequential(nn.Conv2d(input_channels, 8, (3, 3), padding=1),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2),
                                         nn.Conv2d(8, 16, (3, 3), padding=1),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2)
                                        )
        self.linear_layers = nn.Sequential(nn.Linear(input_size * input_size / 4 * 16, 10),
                                           nn.LogSoftmax())
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
import tqdm

all_loss = {}
test_loss = {}

model = ProjectNetwork(...).cuda()
loss_fn = torch.nn.NLLLoss().cuda()

learning_rate = 1e-4
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

nepoch = 10
tqt = tqdm(xrange(nepoch), desc="Train")
for t in tqt:
    j = 0
    ep_loss = 0
    for X, y in train_loader:
        X = Variable(X).type(torch.cuda.FloatTensor)
        y = Variable(y).type(torch.cuda.LongTensor)  ## for classification - vector of longs

        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        ep_loss += loss.data[0]

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        j += 1
    all_loss.append(ep_loss / j)
    tqt.set_description("Loss = {}".format(all_loss[-1]))